# Summer Session II 2025

In [73]:
# The following snippet will automatically reload imported code in helpers.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import csv
from datetime import date, time, timedelta
from typing import Dict, List

from helpers import calendar_table, new_syllabus, show_calendar, Plan, meeting_days, Kind, cut, schedule, merge, serialize, generate_VL_index_page, make_VL_page
from helpers import JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC, MON, WED, FRI, TUES, THUR


## Key Dates

In [75]:
YEAR = 2025

FDOC = date(YEAR, JUN, 23)

GRADESCOPE = "https://www.gradescope.com/"
GSCOPE_DICT = {"": GRADESCOPE}

PRACTICE_PROBLEMS = "/resources/practice/practice-problems.html"

# SSII HOLIDAYS 
HOLIDAYS = {
    date(YEAR, JUL, 4),     # Independence Day
}

# # SPRING HOLIDAYS
# HOLIDAYS = {
#     date(YEAR, JAN, 15),     # MLK JR Day
#     date(YEAR, FEB, 12),     # Well-being Day
#     date(YEAR, FEB, 13),    # Well-being Day
#     date(YEAR, MAR, 28),    # Well-being Day
#     date(YEAR, MAR, 11),    # Spring Break
#     date(YEAR, MAR, 12),    # Spring Break
#     date(YEAR, MAR, 13),    # Spring Break
#     date(YEAR, MAR, 14),    # Spring Break
#     date(YEAR, MAR, 15),    # Spring Break
#     date(YEAR, MAR, 29),    # Univ. Holiday
# }


# # FALL HOLIDAYS 
# HOLIDAYS = {
#     date(YEAR, SEP, 2),     # Labor Day
#     date(YEAR, OCT, 11),    # University Day 
#     date(YEAR, SEP, 3),     # Well-being Day
#     date(YEAR, SEP, 23),    # Well-being Day
#     date(YEAR, OCT, 17),    # Fall Break
#     date(YEAR, OCT, 18),    # Fall Break
#     date(YEAR, NOV, 27),    # Thanksgiving Break
#     date(YEAR, NOV, 28),    # Thanksgiving Break
#     date(YEAR, NOV, 29),    # Thanksgiving Break
# }

DUKE_GAMES = {}

DROP_PERIOD_ENDS = date(YEAR, JUN, 25)
WITHDRAW_PERIOD_ENDS = date(YEAR, JUL, 17)

LDOC = date(YEAR, JUL, 24)

MEETING_DAYS = { MON, TUES, WED, THUR, FRI }

START_TIME = time(9, 30, 0)
END_TIME = time(10, 45, 0)

FINALS = [date(YEAR, JUL, 28)]
FINALS_TIMES = [time(0, 0, 0)]

TIMESPAN = (START_TIME, END_TIME)

## Skeleton

Let's start from the skeleton of a semester with holidays, important semester milestones, Duke games, and lecture days pencilled in.

In [76]:
syllabus: Dict[date, List[Plan]] = new_syllabus(FDOC, LDOC)

for holiday in HOLIDAYS:
    syllabus[holiday].append(Plan("University Holiday", Kind.NA, 0, holiday))

for duke_game in DUKE_GAMES:
    syllabus[duke_game].append(Plan("Duke Game", Kind.NA, 0, duke_game))

syllabus[DROP_PERIOD_ENDS].append(Plan("Drop Period Ends", Kind.NA, 0, DROP_PERIOD_ENDS)) 
syllabus[WITHDRAW_PERIOD_ENDS].append(Plan("Widthdraw Period Ends", Kind.NA, 0, WITHDRAW_PERIOD_ENDS))

MEETINGS = meeting_days(FDOC, LDOC, MEETING_DAYS, HOLIDAYS)
LECTURES = []
for date_idx in range(len(MEETINGS)):
        day = MEETINGS[date_idx]
        lecture = Plan(f"Day {date_idx:02}", Kind.CL, date_idx, day)
        LECTURES.append(lecture)
        syllabus[day].append(lecture)

FN = []
for final_idx in range(len(FINALS)):
    final_time = FINALS_TIMES[final_idx].strftime("%-I:%M%p")
    # MODIFIED FOR SUMMER
    FN.append(schedule(f"Final", syllabus, Kind.FN, 0, FINALS[final_idx]))
    #FN.append(schedule(f"Final @ {final_time}", syllabus, Kind.FN, 0, FINALS[final_idx]))

table = calendar_table(syllabus)
with open('schedule.csv', "w") as csvfile:
    for row in table:
        print(",".join(row))
        csvfile.write(",".join(row) + "\n")


Date,Weekday,Plan
Week 0
Jun-23,Mon,FDOC
,,Day 00
Jun-24,Tue,Day 01
Jun-25,Wed,Drop Period Ends
,,Day 02
Jun-26,Thu,Day 03
Jun-27,Fri,Day 04
Week 1
Jun-30,Mon,Day 05
Jul-01,Tue,Day 06
Jul-02,Wed,Day 07
Jul-03,Thu,Day 08
Jul-04,Fri,University Holiday
Week 2
Jul-07,Mon,Day 09
Jul-08,Tue,Day 10
Jul-09,Wed,Day 11
Jul-10,Thu,Day 12
Jul-11,Fri,Day 13
Week 3
Jul-14,Mon,Day 14
Jul-15,Tue,Day 15
Jul-16,Wed,Day 16
Jul-17,Thu,Widthdraw Period Ends
,,Day 17
Jul-18,Fri,Day 18
Week 4
Jul-21,Mon,Day 19
Jul-22,Tue,Day 20
Jul-23,Wed,Day 21
Jul-24,Thu,LDOC
,,Day 22
Week 5
Jul-28,Mon,Final


## Quizzes

In [77]:
# Update the following tuple to reflect meeting days quizzes are held on.
# QZ_MEETING_DAYS = (7, 13, 25, 31, 37)
QZ_MEETING_DAYS = (4, 8, 13, 18) # Just put somewhat random dates here! - Izzi

quizzes = cut(syllabus, FDOC, LDOC)

QZ: Plan = list()


for i in range(len(QZ_MEETING_DAYS)):
    quiz_date = MEETINGS[QZ_MEETING_DAYS[i]]
    QZ.append(schedule(f"Quiz {i}", quizzes, Kind.QZ, i, quiz_date, quiz_date))
#show_calendar(quizzes)

In [78]:

#TODO: For Fa25, change back to start last unit with qz[3]
UNIT_DATES = [
    { "start": FDOC, "end": QZ[0].date },
    { "start": QZ[0].date + timedelta(days=1), "end": QZ[1].date },
    { "start": QZ[1].date + timedelta(days=1), "end": QZ[2].date },
    #{ "start": QZ[2].date + timedelta(days=1), "end": QZ[3].date },
    { "start": QZ[2].date + timedelta(days=1), "end": FN[-1].date }
]

unit: list[dict[date, list[Plan]]] = []
for i in range(len(UNIT_DATES)):
    unit_dates = UNIT_DATES[i]
    unit.append(cut(syllabus, UNIT_DATES[i]["start"], UNIT_DATES[i]["end"]))


In [79]:
# Collections for various components of the course
CL: list[Plan] = []
LS: list[Plan] = []
EX: list[Plan] = []
CQ: list[Plan] = []
RD: list[Plan] = []
VL: list[Plan] = []

# These indices are kind of a strange idea at first glance. The problem we are hoping to solve is
# keeping track of index numbers (to count up indices of each course component), but we'd like 
# to be able to reexecute unit cells below. So what we do is keep an index for each of the units.
# When subsequent units begin, they start from the count of the previous range (and 0 in the first).
# This way, you can reexecute cells below multiple times without having to reinitialize these indices.
# This is related to the trick of cutting / pasting the lesson plans.
CL_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
LS_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
EX_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
CQ_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
RD_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
VL_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]

LS_due_delta: timedelta = timedelta(days=0)
VL_due_delta: timedelta = timedelta(days=1)
CQ_due_delta: timedelta = timedelta(days=0)
EX_due_delta: timedelta = timedelta(days=3)
RD_due_delta: timedelta = timedelta(days=20)

LS_VIDEOS = []
LS_links = {}

LDOC = MEETINGS[len(MEETINGS)-1]
# Auto populate LS
for idx in range(100):
    LS.append(schedule("", unit[0], Kind.LS, LS_idxs[0], LDOC))


# Functions to Create Assignments

In [80]:
def add_LS(name: str, links: dict, due_date: timedelta = LS_due_delta):
    global LS_idxs
    global LS
    global date_ctx
    global u
    global LS_VIDEOS
    global LS_links
    LS[LS_idxs[u]] = (schedule(name, unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + due_date))
    LS[LS_idxs[u]].links = links
    LS_idxs[u] += 1
    #LS_VIDEOS.append([name, links])
    #LS_links[name] = links
        

def add_EX(name: str, links: dict, due_date: timedelta = EX_due_delta):
    global EX_idxs
    global EX
    global date_ctx
    global u
    EX.append(schedule(name, unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + due_date))
    EX[EX_idxs[u]].links = links
    EX_idxs[u] += 1
    
def add_CL(name: str, links: dict):
    global CL_idxs
    global CL
    global date_ctx
    global u
    global LS_links
    CL.append(schedule(name, unit[u], Kind.CL, CL_idxs[u], date_ctx))
    CL[CL_idxs[u]].links = links
    CL_idxs[u] += 1
    LS_links[name] = links

    
def add_CQ(name: str, links: dict, due_date = timedelta(days=0)):
    global CQ_idxs
    global CQ
    global date_ctx
    global u
    CQ.append(schedule(name, unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + timedelta(days=0) + due_date))
    CQ[CQ_idxs[u]].links = links
    CQ_idxs[u] += 1
    

    

In [81]:
u = 0  # Current unit index

# Reset stateful counters and unit schedule
CL_idxs[u] = 0 # First unit starts CL from 0
CL = CL[:CL_idxs[u]] # Reset CL list - redundant in first unit, but keeping for consistency
VL_idxs[u] = 0 # First unit starts VL from 0
VL = VL[:VL_idxs[u]] # Reset VL list - redundant in first unit, but keeping for consistency
LS_idxs[u] = 0 # First unit starts LS from 0
# LS = LS[:LS_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
EX_idxs[u] = 0 # First unit starts EX from 0
EX = EX[:EX_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
CQ_idxs[u] = 0 # First unit starts CQ from 0
# CQ = CQ[:CQ_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
RD_idxs[u] = 0 # First unit starts CQ from 0
RD = RD[:RD_idxs[u]] # Reset RD list - redundant in first unit, but keeping for consistency
unit[u] = cut(syllabus, UNIT_DATES[u]["start"], UNIT_DATES[u]["end"])

# Date Context is the current Meeting # we're anchoring off of. We make this a variable to make
# moving around scheduled items easier.

In [82]:
# Week 0
date_ctx = MEETINGS[0]

day_0 = {"Welcome Video": "https://youtu.be/3qqaCsEVzng", 
         "Welcome Slides": "/static/slides/CL00.pdf",
         "Setup Part 0: Update Operating System" : "/resources/setup/os-update.html",
         "Setup Part 1: Install Needed Software" : "/resources/setup/software.html",
         "Setup Part 2: Setup your Workspace" : "/resources/setup/workspace.html",
         }

add_CL("Welcome to COMP110", day_0)



add_LS("Syllabus", GSCOPE_DICT)


date_ctx = MEETINGS[1]

day_1 = { "Running a Program (Video)": "https://youtu.be/M1FeIzICA9A",
         "Objects and Data Types (Video)": "https://www.youtube.com/watch?v=6GxkRgIze-c",
         "Object and Data Types (Lecture Notes)": "/lessons/objects_data_types.html"  
}

add_CL("Objects and Data Types", day_1)

add_LS("Running A Program", GSCOPE_DICT)

add_LS("Objects and Data Types", GSCOPE_DICT)

add_EX("Hello World", {"": "/exercises/ex00_hello_world.html"})

# In LS00, include links to resources, support, and syllabus

date_ctx = MEETINGS[2]

day_2 = {
    "Expressions Video" : "/",
    "Functions Video": "/",
    "Memory Diagrams Video": "/",
    "Slides": "/"
}

add_CL("Expressions, Functions, and Memory Diagrams", day_2)

add_LS("Expressions", GSCOPE_DICT)

add_LS("Functions", GSCOPE_DICT)

add_CQ("Memory Diagrams Practice", GSCOPE_DICT)



date_ctx = MEETINGS[3]

add_CQ("Writing Functions", {"": "/cqs/CQ00-functions.html"})



In [83]:

syllabus = merge(syllabus, quizzes)
for u in unit:
    syllabus = merge(syllabus, u)
serializable_syllabus = serialize(syllabus)
show_calendar(syllabus)

Date,Weekday,Plan
Week 0,,
Jun-23,Mon,FDOC
,,Day 00
,,Welcome to COMP110
,,Out: Syllabus
,,Due: Syllabus
Jun-24,Tue,Day 01
,,Objects and Data Types
,,Out: Running A Program
,,Out: Objects and Data Types


In [84]:
syllabus_copy = syllabus.copy()